In [12]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("\tmp\data", one_hot=True)

Extracting 	mp\data/train-images-idx3-ubyte.gz
Extracting 	mp\data/train-labels-idx1-ubyte.gz
Extracting 	mp\data/t10k-images-idx3-ubyte.gz
Extracting 	mp\data/t10k-labels-idx1-ubyte.gz


In [20]:
learning_rate = 0.0002
batch_size = 128
num_steps = 10000
display_steps = 1000

image_dim = 28*28
hidden_gen = 256
hidden_dis = 256
noise_dim = 100

def glorot_init(shape):
    return tf.random_normal(shape, stddev=1. / tf.sqrt(shape[0] / 2.))

In [21]:
weights = {
    'gen_h1': tf.Variable(glorot_init([noise_dim, hidden_gen])),
    'gen_out': tf.Variable(glorot_init([hidden_gen, image_dim])),
    'dis_h1': tf.Variable(glorot_init([image_dim, hidden_dis])),
    'dis_out': tf.Variable(glorot_init([hidden_dis, 1]))
}

biases = {
    'gen_b1': tf.Variable(glorot_init([hidden_gen])),
    'gen_out': tf.Variable(glorot_init([image_dim])),
    'dis_b1': tf.Variable(glorot_init([hidden_dis])),
    'dis_out': tf.Variable(glorot_init([1]))
}

def generator(noise):
    layer1 = tf.matmul(noise, weights['gen_h1']) + biases['gen_b1']
    layer1 = tf.nn.relu(layer1)
    out = tf.matmul(layer1, weights['gen_out']) + biases['gen_out']
    out = tf.nn.sigmoid(out)
    return out

def discriminator(image):
    layer1 = tf.matmul(image, weights['dis_h1']) + biases['dis_b1']
    layer1 = tf.nn.relu(layer1)
    out = tf.matmul(layer1, weights['dis_out']) + biases['dis_out']
    out = tf.nn.sigmoid(out)
    return out

input_gen = tf.placeholder(tf.float32, [None, noise_dim])
input_dis = tf.placeholder(tf.float32, [None, image_dim])

gen_sample = generator(input_gen)

dis_real = discriminator(input_dis)
dis_fake = discriminator(gen_sample)

loss_gen = -tf.reduce_mean(tf.log(dis_fake))
loss_dis = -tf.reduce_mean(tf.log(dis_real) + tf.log(1. - dis_fake))


optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_dis = tf.train.AdamOptimizer(learning_rate=learning_rate)

gen_vars = [weights['gen_h1'], weights['gen_out'],
            biases['gen_b1'], biases['gen_out']]
dis_vars = [weights['dis_h1'], weights['dis_out'],
            biases['dis_b1'], biases['dis_out']]

train_gen = optimizer_gen.minimize(loss_gen, var_list=gen_vars)
train_dis = optimizer_dis.minimize(loss_dis, var_list=dis_vars)

init = tf.global_variables_initializer()

In [22]:
with tf.Session() as sess:
    
    sess.run(init)
    
    for step in range(1, num_steps):
        batch_x, _ = mnist.train.next_batch(batch_size)
        
        z = np.random.uniform(-1., 1., size=[batch_size, noise_dim])
        
        feed_dict = {input_gen: z, input_dis: batch_x}
        _,_,gl,dl = sess.run([train_gen, train_dis, loss_gen, loss_dis], feed_dict = feed_dict)
        
        if step % display_steps == 0 or step == 1:
            print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (step, gl, dl))

Step 1: Generator Loss: 0.023015, Discriminator Loss: 3.869399
Step 1000: Generator Loss: 2.524072, Discriminator Loss: 0.280537
Step 2000: Generator Loss: 3.537890, Discriminator Loss: 0.066817
Step 3000: Generator Loss: 2.323431, Discriminator Loss: 0.239935
Step 4000: Generator Loss: 2.622757, Discriminator Loss: 0.205005
Step 5000: Generator Loss: 3.138986, Discriminator Loss: 0.331748
Step 6000: Generator Loss: 3.143493, Discriminator Loss: 0.219871
Step 7000: Generator Loss: 2.682419, Discriminator Loss: 0.237796
Step 8000: Generator Loss: 3.414693, Discriminator Loss: 0.149640
Step 9000: Generator Loss: 3.051258, Discriminator Loss: 0.207871
